In [91]:
import numpy as np
import matplotlib.pyplot as plt
import random

# defining simple lattice functions to place n particles in a box of length L 

def simple_lattice_2D(L,n):
     positions = []
     N = int(np.sqrt(n))
     separation = L / N
     for i in range(0,N):
            for j in range(0,N):
                x = (i + 0.5) * separation
                y = (j + 0.5) * separation
                positions.append(list([x,y]))
     return positions


def simple_lattice_3D(L,n):
    positions = []
    N = int(np.cbrt(n))
    separation = L / N
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                x = (i + 0.5) * separation
                y = (j + 0.5) * separation
                z = (k + 0.5) * separation
                positions.append((x,y,z))
    return positions

def particle_diameter_3D(L, n, vol_fraction):
    volume_per_particle = (L**3 * vol_fraction) / n
    diameter = 2*(3 * volume_per_particle / (4 * np.pi))**(1/3)
    return diameter
def particle_diameter_2D(L,n, area_fraction):
    area_per_particle = (L**2 * area_fraction) / n
    diameter = 2*(area_per_particle / np.pi)**(1/2)
    return diameter

x_coords, y_coords = zip(*simple_lattice_2D(L,100))
plt.figure()
plt.plot(x_coords, y_coords, 'o')
plt.savefig('simple_lattice_2D')

def monte_carlo_2D_np(positions, displacement, L, diameter):
    positions = np.array(positions, dtype=float)  # ensure numeric array
    N = len(positions)
    
    # Pick a random particle
    particle = random.randint(0, N - 1)
    
    # Copy positions for trial move
    new_positions = positions.copy()
    
    # Random displacement along x or y
    axis = random.randint(0, 1)       # 0=x, 1=y
    direction = 1 if random.randint(0, 1) == 0 else -1
    new_positions[particle, axis] += direction * displacement
    
    # Apply periodic boundary conditions
    new_positions[particle] %= L
    
    # Compute distances from the moved particle to all others
    diffs = new_positions - new_positions[particle]         # shape (N, 2)
    
    # Minimum image convention for periodic boundaries
    diffs = diffs - L * np.round(diffs / L)
    
    distances = np.linalg.norm(diffs, axis=1)
    distances[particle] = np.inf  # ignore self-distance
    
    # Check overlap
    if np.any(distances < diameter):
        return positions  # reject move
    else:
        return new_positions  # accept move
    
L = 4
n = 4 
vol_fraction = 0.1
no_density = n / (L**2)
diameter = particle_diameter(L, n, vol_fraction)



In [92]:
# create stepwise function to move random particle in random direction by displacement amount specified
#also account for the periodic boundary conditions inside the movement 
import random

def monte_carlo_2D(positions, displacement, L, diameter):
    positions = np.array(positions, dtype=float)  # ensure numeric array
    N = len(positions)
    
    # Pick a random particle
    particle = random.randint(0, N - 1)
    
    # Copy positions for trial move
    new_positions = positions.copy()
    
    # Random displacement along x or y
    axis = random.randint(0, 1)       # 0=x, 1=y
    direction = 1 if random.randint(0, 1) == 0 else -1
    new_positions[particle, axis] += direction * displacement
    
    # Apply periodic boundary conditions
    new_positions[particle] %= L
    
    # Compute distances from the moved particle to all others
    diffs = new_positions - new_positions[particle]         # shape (N, 2)
    
    # Minimum image convention for periodic boundaries
    diffs = diffs - L * np.round(diffs / L)
    
    distances = np.linalg.norm(diffs, axis=1)
    distances[particle] = np.inf  # ignore self-distance
    
    # Check overlap
    if np.any(distances < diameter):
        return positions  # reject move
    else:
        return new_positions  # accept move



In [137]:
#list of provided parameters - good starting point


L = 1
n = 100
vol_fraction = 0.72
no_density = n / (L**2)
diameter = particle_diameter_2D(L, n, vol_fraction)
pos_0 = simple_lattice_2D(L,n)

pos_new = monte_carlo_2D(pos_0, 0.0001, L, diameter)

if np.array_equal(pos_new, pos_0):
    print("Move rejected")
else:
    print("Move accepted")
  
#particles free to move with monte carlo function for 2D
#code to find the optimum displacement value to achieve 25% - 50% acceptance rate
acceptance_count = 0
pos_ref = pos_0
for i in range(0,1000):
    pos_new = monte_carlo_2D(pos_ref, 0.004254, L, diameter)
    if np.array_equal(pos_new, pos_ref):
        pass #move rejected
    else:
        acceptance_count += 1 #move accepted
        pos_ref = pos_new
print(acceptance_count)


Move accepted
0
